In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

import multirtd.utils as utils
from multirtd.zonotope import Zonotope
from multirtd.LPM import LPM

%load_ext autoreload
%autoreload 2

In [ ]:
# System
dt = 0.1
n_s = 4  # State dimension
n_c = 2  # Control dimension
n_m = 2  # Measurement dimension

A = np.eye(n_s)  # Dynamics matrix
A[0:2,2:4] = dt*np.eye(2)
B = np.array([[dt**2/2, 0],  # Control matrix
              [0, dt**2/2],
              [dt, 0],
              [0, dt]])
C = np.array([[1, 0, 0, 0],  # Measurement matrix
              [0, 1, 0, 0]])

Q_LQR = np.eye(n_s)
R_LQR = np.eye(n_c)
K = utils.dlqr_calculate(A, B, Q_LQR, R_LQR)

Q = 0.01

In [ ]:
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
lpm = LPM(LPM_file)

LPM FRS Computation

In [ ]:
# Peak velocity space
v_max = 3
c_pk = np.zeros((2,1))
G_pk = v_max * np.eye(2)
V_pk = Zonotope(c_pk, G_pk)

# Zeros zonotope
Z0 = Zonotope(np.zeros((2,1)), np.zeros((2,2)))

N = len(lpm.time)  # Trajectory length
P_LPM = lpm.P_mat.T

PRS = N * [None]

In [ ]:
# Fixed initial conditions
v_0 = np.array([4,2])
a_0 = np.array([5,5])
k_0 = np.vstack((v_0, a_0))

p_from_ic = P_LPM[:,1:3] @ k_0

fig, ax = plt.subplots(figsize=(10,10))

v_pk = np.array([1,1])[:,None]

for i in range(N):
    P_from_vpk = P_LPM[i,2] * V_pk
    PRS[i] = P_from_vpk.augment(V_pk) + np.vstack((p_from_ic[i,:][:,None], np.zeros((2,1))))
    PRS[i].view([0,1]).plot(ax)

    PRS_slc = PRS[i].slice([2,3], v_pk)
    PRS_slc.view([0,1]).plot(ax, color='r')

In [ ]:
from multirtd.reachability import compute_FRS



Test slicing

In [ ]:
Z = PRS[-1]
c = Z.c; G = Z.G

slice_dim = [2,3]
slice_pt = np.array([0.3,0.3])[:,None]

In [ ]:
PRS[-1].slice([2,3], [0,0])

In [ ]:
slice_idx = []

for i in range(len(slice_dim)):
    myidxs = np.nonzero(G[slice_dim[i]])[0]  # Find non-zero generators in slice dimension
    if len(myidxs) != 1:
        if len(myidxs) == 0:
            print('No generator for slice index')
        else:
            print('Multiple generators for slice index')
    slice_idx.append(myidxs[0])

In [ ]:
slice_c = c[slice_dim]
slice_G = G[slice_dim,:][:,slice_idx]
slice_lambda = np.linalg.solve(slice_G, slice_pt - slice_c)  # Calculate coefficients for slice
if slice_lambda.shape[1] > 1:
    print('slice_lambda is not 1D')
if np.any(np.abs(slice_lambda ) > 1):
    print('Slice point is outside of bound of zonotope')

newG = G
newG = np.delete(newG, slice_idx, axis=1)
newc = c+ G[:,slice_idx] @ slice_lambda

In [ ]:
newc

In [ ]:
newG